<a href="https://colab.research.google.com/github/valkryhx/Firefly/blob/master/gptq_lora_baichuan7b_vicuna_v2_%E6%94%AF%E6%8C%81%E5%88%86%E5%B8%83%E5%BC%8F%E8%AE%AD%E7%BB%83_%E6%95%88%E6%9E%9C%E5%A5%BD%E5%83%8F%E5%8F%AF%E4%BB%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/valkryhx/Firefly

Cloning into 'Firefly'...
remote: Enumerating objects: 664, done.
remote: Counting objects: 100% (499/499), done.
remote: Compressing objects: 100% (300/300), done.
remote: Total 664 (delta 285), reused 309 (delta 199), pack-reused 165
Receiving objects: 100% (664/664), 6.39 MiB | 16.15 MiB/s, done.
Resolving deltas: 100% (370/370), done.


In [ ]:
%cd Firefly/
!pip install -r requirements.txt

/content/Firefly
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-epucao1p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-epucao1p
  Resolved https://github.com/huggingface/peft.git to commit e06d94ddeb6c70913593740618df76908b918d66
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
pwd

'/content/Firefly'

In [ ]:
%cd Firefly/

[Errno 2] No such file or directory: 'Firefly/'
/content/Firefly


# gptq + lora 训练： 从base model开始

In [ ]:
!git pull --all --force
! python train_gptq_lora.py  --train_args_file train_args/qlora/baichuan-7b-sft-qlora.json \
              --output_dir  output/baichuan-gptq-v1

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 677 bytes | 677.00 KiB/s, done.
From https://github.com/valkryhx/Firefly
   9bd5a6f..ae83e96  master     -> origin/master
Updating 9bd5a6f..ae83e96
Fast-forward
 test_gptq_lora_adapter.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)
Exception ignored in: <function _xla_gc_callback at 0x7c9d7e7752d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
[2023-07-21 09:09:48,078] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-07-21 09:09:50.381578: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-21 0

# gptq + lora 继续训练：从base model挂载上次的lora adapter 继续lora训练

#  代码中居然是str类型？不能写bool 否则只要传参就是True 真是玄学
# parser.add_argument("--ddp_find_unused_parameters", type=str, default=False)
# 传参 --ddp_find_unused_parameters False

In [ ]:
!git pull --all --force
# !torchrun --nproc-per-node=1
!deepspeed --num_gpus=1 train_gptq_lora.py  --train_args_file train_args/qlora/baichuan-7b-sft-qlora.json \
              --peft_path  output/baichuan-gptq-v1/final   \
              --output_dir  output/baichuan-gptq-v2 \
              --ddp_find_unused_parameters False

Fetching origin
Already up to date.
[2023-07-21 08:09:30,277] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-07-21 08:09:36.950524: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-07-21 08:09:39,547] [WARNING] [runner.py:196:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2023-07-21 08:09:39,562] [INFO] [runner.py:555:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None train_gptq_lora.py --train_args_file train_args/qlora/baichuan-7b-sft-qlora.json --peft_path output/baichuan-gptq-v1/final --output_dir output/baichuan-gptq-v2 --ddp_find_unused_parameters False
[2023-07-21 08:09:41,985] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-07-21 08:09:45.593910: W tensorflow

In [ ]:
pwd

'/content/Firefly'

In [ ]:
!git pull --all --force
!python test_gptq_lora_adapter.py \
     --peft_path output/baichuan-gptq-v2/final \
     --base_model_name_or_path fireballoon/baichuan-vicuna-chinese-7b-gptq

Fetching origin
Already up to date.
[2023-07-21 09:27:50,372] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-07-21 09:27:53.762935: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-21 09:27:57.150 | INFO     | __main__:main:58 - args.use_safetensors= False , <class 'bool'>
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
PeftModelForCausalLM(
  (base_model): GPTQLoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(64000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): F